# WorkFlow

* Intro to Tensors
* Getting info from Tensors
* Manipulating Tensors
* Tensors and Numpy
* Using @tf.function (speed up regular python fucntion)
* Using GPU/TPU w/TensorFlow
* Excercise


### Intro to `Tensors`

In [1]:
# import tensorflow

import tensorflow as tf
print(tf.__version__)

2022-12-05 23:37:42.547925: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0


In [2]:
# creating tensors with tf.contant()
scalar = tf.constant(7)
scalar
# has an empty shape
# tf.constant? ceates a constant tensor from a tensor-like object

2022-12-05 23:37:45.550243: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# check for number of dimensions
scalar.ndim

0

In [4]:
# create a vector
vector = tf.constant([10,10]) # pass a python list to constant function
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# check dimension of vector
vector.ndim

1

In [6]:
# create a matrix
matrix = tf.constant([
    [10,7],
    [7,10]
])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# check for dimension
matrix.ndim

2

In [8]:
another_matrix = tf.constant([[1.,4.],
[3.,5.],
[9., 12.]], dtype=tf.float16) # reduce data size occuiance on disk

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[ 1.,  4.],
       [ 3.,  5.],
       [ 9., 12.]], dtype=float16)>

In [9]:
another_matrix.ndim #number of elements in a shape

2

In [10]:
# create another tensor
tensor = tf.constant(
    [[[1,2,3,],
        [4,5,6]],
        [[7,8,9],
        [10,11,12]],
        [[13,14,15],
        [16,17,18]]]
)
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

### Creating Tensors with `tf.Variable`

In [17]:
changable_tensor = tf.Variable([10,7])
unchangable_tensor = tf.constant([10,7])

changable_tensor, unchangable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [13]:
# lets chnag one of the elements in a chnagable tensor
changable_tensor[0] = 13

TypeError: 'ResourceVariable' object does not support item assignment

In [18]:
# trying assign
changable_tensor[1].assign(15)

<tf.Variable 'UnreadVariable' shape=(2,) dtype=int32, numpy=array([10, 15], dtype=int32)>

In [15]:
# changing constant
unchangable_tensor[0].assign(1)

AttributeError: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'assign'